In [2]:
import pandas as pd

# Load the dataset
file_path = 'C:\\Users\\HP\\OneDrive\\Desktop\\Machine Learning Projects\\Location_based_analysis\\data\\Dataset .csv'
df = pd.read_csv(file_path)




In [3]:
# Display the first few rows of the dataframe
df.head(3)

,Restaurant ID,Restaurant Name,Country Code,City,Address,Locality,Locality Verbose,Longitude,Latitude,Cuisines,...,Currency,Has Table booking,Has Online delivery,Is delivering now,Switch to order menu,Price range,Aggregate rating,Rating color,Rating text,Votes
0,6317637,Le Petit Souffle,162,Makati City,"Third Floor, Century City Mall, Kalayaan Avenu...","Century City Mall, Poblacion, Makati City","Century City Mall, Poblacion, Makati City, Mak...",121.027535,14.565443,"French, Japanese, Desserts",...,Botswana Pula(P),Yes,No,No,No,3,4.8,Dark Green,Excellent,314
1,6304287,Izakaya Kikufuji,162,Makati City,"Little Tokyo, 2277 Chino Roces Avenue, Legaspi...","Little Tokyo, Legaspi Village, Makati City","Little Tokyo, Legaspi Village, Makati City, Ma...",121.014101,14.553708,Japanese,...,Botswana Pula(P),Yes,No,No,No,3,4.5,Dark Green,Excellent,591
2,6300002,Heat - Edsa Shangri-La,162,Mandaluyong City,"Edsa Shangri-La, 1 Garden Way, Ortigas, Mandal...","Edsa Shangri-La, Ortigas, Mandaluyong City","Edsa Shangri-La, Ortigas, Mandaluyong City, Ma...",121.056831,14.581404,"Seafood, Asian, Filipino, Indian",...,Botswana Pula(P),Yes,No,No,No,4,4.4,Green,Very Good,270


In [4]:
import folium
from folium.plugins import HeatMap

# Create a map centered around a specific location
m = folium.Map(location=[20.5937, 78.9629], zoom_start=5)  # Centered on India

# Add points to the map
for index, row in df.iterrows():
    folium.Marker([row['Latitude'], row['Latitude']], popup=row['Restaurant Name']).add_to(m)

# Save the map as an HTML file
m.save('../static/restaurants_map.html')


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9551 entries, 0 to 9550
Data columns (total 21 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Restaurant ID         9551 non-null   int64  
 1   Restaurant Name       9551 non-null   object 
 2   Country Code          9551 non-null   int64  
 3   City                  9551 non-null   object 
 4   Address               9551 non-null   object 
 5   Locality              9551 non-null   object 
 6   Locality Verbose      9551 non-null   object 
 7   Longitude             9551 non-null   float64
 8   Latitude              9551 non-null   float64
 9   Cuisines              9542 non-null   object 
 10  Average Cost for two  9551 non-null   int64  
 11  Currency              9551 non-null   object 
 12  Has Table booking     9551 non-null   object 
 13  Has Online delivery   9551 non-null   object 
 14  Is delivering now     9551 non-null   object 
 15  Switch to order menu 

In [6]:
# Handle potential missing values
df['Aggregate rating'].fillna(df['Aggregate rating'].mean(), inplace=True)
df['Price range'].fillna(df['Price range'].mode()[0], inplace=True)
df['Cuisines'].fillna('Unknown', inplace=True)

# Group by city
grouped = df.groupby('City').agg({
    'Aggregate rating': 'mean',
    'Average Cost for two': 'mean',
    'Cuisines': lambda x: x.mode().iloc[0] if not x.mode().empty else 'Unknown',  # Handle empty mode results
    'Price range': 'mean'
}).reset_index()

# Display the grouped data
grouped.head()


C:\Users\HP\AppData\Local\Temp\ipykernel_10068\4090787323.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['Aggregate rating'].fillna(df['Aggregate rating'].mean(), inplace=True)
C:\Users\HP\AppData\Local\Temp\ipykernel_10068\4090787323.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves

,City,Aggregate rating,Average Cost for two,Cuisines,Price range
0,Abu Dhabi,4.300000,182.000000,American,3.300000
1,Agra,3.965000,1065.000000,"North Indian, Mughlai",2.650000
2,Ahmedabad,4.161905,857.142857,"Cafe, American, Continental, Armenian, Fast Food",2.571429
3,Albany,3.555000,19.750000,Unknown,1.700000
4,Allahabad,3.395000,517.500000,"North Indian, Chinese",2.650000


In [7]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
import pickle


In [8]:
# Prepare the data
X = df[['Latitude', 'Longitude', 'Average Cost for two', 'Price range']]  # Example features
y = df['Aggregate rating']

In [9]:
# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [10]:
# Train the model
model = LinearRegression()
model.fit(X_train, y_train)

LinearRegression()

In [11]:
# Evaluate the model
predictions = model.predict(X_test)
mse = mean_squared_error(y_test, predictions)
print(f'Mean Squared Error: {mse}')


Mean Squared Error: 1.8195428471848305


In [12]:
model.score(X_train, y_train)*100

20.655352694156505

In [13]:
# Save the model
with open('../model/restaurant_model.pkl', 'wb') as file:
    pickle.dump(model, file)

In [17]:
df.columns

Index(['Restaurant ID', 'Restaurant Name', 'Country Code', 'City', 'Address',
       'Locality', 'Locality Verbose', 'Longitude', 'Latitude', 'Cuisines',
       'Average Cost for two', 'Currency', 'Has Table booking',
       'Has Online delivery', 'Is delivering now', 'Switch to order menu',
       'Price range', 'Aggregate rating', 'Rating color', 'Rating text',
       'Votes'],
      dtype='object')